租房信息已经有了，为了能对北京目前的租房市场有个直观认识，我对数据进行深度分析，并进行可视化展示

从分析结果中，我得到了哪些位置房源多、各区租房平均价格以及心仪价格地理位置分布等重要信息，为帮助我租房提供重要依据

下面带大家一起看一下整个分析过程

### 1.分析各行政区房源数量及单价

In [ ]:
import pandas as pd
beijing_daname=['朝阳区', '丰台区', '海淀区', '大兴区', '通州区', '昌平区', '东城区', '西城区', '顺义区']
data=pd.read_csv('租房数据加经纬度.csv',encoding='gbk')
areas=list(set(list(data['行政区'])))
area_sums={}
for area in areas:
    area_sums[area]=list(data['行政区']).count(area)
from pyecharts import options as opts
from pyecharts.charts import Bar
import random
hotel_num=[area_sums[i] for i in beijing_daname]
bar = (
    Bar()
    .add_xaxis(beijing_daname)
    .add_yaxis("", hotel_num)
    .set_global_opts(title_opts=opts.TitleOpts(title="北京各区房源数量"))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=True),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
                opts.MarkLineItem(type_="average", name="平均值"),
            ]
        ),
    )
)
bar.render_notebook()

从上图可以得出以下结论：

朝阳区的房源数量最多，有1877套  
顺义区的房源数量最少，有272套  
9个区平均房源数量为611套。

各城区房源单价情况（每平米单价*30平米为例）

In [ ]:
unit_price={}
for i in list(data.groupby('行政区')):
    if i[0] in beijing_daname:
        unit_price[i[0]]=int(i[1]['价格'].sum()/i[1]['面积'].sum())*30
unit_price
bar = (
    Bar()
    .add_xaxis(list(unit_price.keys()))
    .add_yaxis("", [unit_price[i] for i in list(unit_price.keys())])
    .set_global_opts(title_opts=opts.TitleOpts(title="北京各区租房均价（每平米单价*30平米为例）"))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=True),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
                opts.MarkLineItem(type_="average", name="平均值"),
            ]
        ),
    )
)
bar.render_notebook()

以30平米为例：  
西城区的住房价格最高，为4350元  
通州区的租房价格最低，为1620元  
价格差距还是很大的。

### 2.分析分析各户型占比及价格分布

In [ ]:
layouts=list(set(data['户型']))
layout=data.loc[:,'户型'].value_counts()
from pyecharts import options as opts
from pyecharts.charts import Pie
print(list(layout.index)[:10])
values=[int(i) for i in list(layout.values)[:10]]
pie = (
    Pie()
   .add(
        "",
        [(i,j)for i,j in zip(list(layout.index)[:10],values)],
        radius=["30%", "75%"],
        center=["40%", "50%"],
        rosetype="radius",
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="北京市各区出租房户型占比"),
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="85%", orient="vertical"),)
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c},{d}%"))
)
pie.render_notebook()

由于户型种类比较多，所以我只选了前10种，从图中可以看出，房源主流是1室1厅1卫，占比41.86%,其次分别是是2室1厅1卫、1室0厅1卫，各占比30.58%和11.02%

In [ ]:
cut_n=list(range(0,12000,1000))
income=pd.cut(data["价格"],cut_n)
price_cut=data['价格'].groupby(income).count()
index=list(price_cut.index)
index=[str(i) for i in list(price_cut.index)]
values=[int(i) for i in list(price_cut.values)]
pie = (
    Pie()
   .add(
        "",
        [(i,j)for i,j in zip(index,values)],
        radius=["30%", "75%"],
        center=["40%", "50%"],
        rosetype="radius",
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="北京市各区出租房户型占比"),
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="85%", orient="vertical"),)
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:  {c}, {d}%"))
)
pie.render_notebook()

目前，主流的租房价格在3000至6000元，占比50%左右，最便宜有1000元以下的,位置相对较偏，且面积在20平以内；贵的有1万多的,这种一般面积在100平以上，位置在主城区

### 3.房子位置分布

根据上述分析情况和我的预算，我决定在朝阳区找一套合适的房子，取出价格在4000至6000元的数据，另存表格，并将表格导入水经注地图下载器中

In [ ]:
com_data=data[data['价格'].le(6000)]
com_data=com_data[com_data['价格'].ge(3000)]
com_data=com_data[com_data['行政区']=='朝阳区']
com_data.to_csv('心仪房子.csv',encoding='gbk')

结果展示如下:

下面红色数字为租房价格，这样找起来就更方便了